In [2]:
pip install selenium

   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ------------------------ --------------- 5.8/9.4 MB 27.1 MB/s eta 0:00:01
   ---------------------------------------- 9.4/9.4 MB 25.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0
Note: you may need to restart the kernel to use updated packages.


In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, pandas as pd, os, pickle

u2n=lambda x:float(x.replace(",","")[:-1])*(1000**{"K":1,"M":2,"B":3}[x[-1]]) if int(x[-1],36)>9 else int(x.replace(",",""))

df=pd.read_csv("C:/Users/망고/Documents/1000billion/sampled_label.csv").iloc[5:10]
df["id"]=df["id"].apply(lambda x:x[1:])
# 🔧 Chrome 설정
options = Options()
options.add_argument("--window-size=1920x1080")
options.add_argument("--incognito")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://twitter.com/home")

WebDriverWait(driver, 30).until( EC.presence_of_element_located((By.XPATH, '//div[@aria-label="Timeline: Your Home Timeline"]')) )
print("✅ 로그인 완료!")

results = []

for i in range(len(df)):
    try:
        driver.get(f"https://twitter.com/i/user/{df.iloc[i]["id"]}")

        time.sleep(3)

        current_url = driver.current_url
        username = current_url.split("/")[-1]

        # 기본 정보 수집
        is_private=0
        try: display_name = driver.find_element(By.XPATH, '//div[@data-testid="UserName"]//span[1]').text
        except: display_name = ""

        try: bio = driver.find_element(By.XPATH, '//div[@data-testid="UserDescription"]').text
        except: bio = ""

        try: profile_pic = driver.find_element(By.XPATH, '//img[@alt="Image"]').get_attribute("src")
        except: profile_pic = ""
            
        try:
            followers = driver.find_element(By.XPATH, '//a[contains(@href,"/verified_followers")]//span[1]/span').text
        except:
            is_private=1
            try: followers = driver.find_element(By.XPATH, '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/div[3]/div/div/div[1]/div[2]/div[5]/div[2]/button/span[1]/span').text
            except: followers = ""
        followers=u2n(followers)
        
        try: following = driver.find_element(By.XPATH, '//a[contains(@href,"/following")]//span[1]/span').text
        except:
            try: following = driver.find_element(By.XPATH, '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/div[3]/div/div/div[1]/div[2]/div[5]/div[1]/button/span[1]/span').text
            except: following = ""
        following=u2n(following)

        try: posts = driver.find_element(By.XPATH, '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/div[1]/div[1]/div/div/div/div/div/div[2]/div/div').text[:-6]
        except: posts = ""
        posts=u2n(posts)

        nums_in_username = sum(c.isdigit() for c in username)
        nums_in_fullname = sum(c.isdigit() for c in display_name)
        external_url = "http" in bio
        
        results.append({
            "user_id": df.iloc[i]["id"],
            "nums/lenth username": nums_in_username/len(username),
            "fullname words": len(display_name.split()),
            "nums/lenth fullname": nums_in_fullname/len(display_name),
            "name==username": int(display_name.lower() == username.lower()),
            "profile_pic": 1,
            "description length": len(bio),
            "external URL": int(external_url),
            "private": int(is_private),
            "#posts": posts,
            "#followers": followers,
            "#follows": following,
            "fake": df.iloc[i]["label"]
        })

        print(f"✅ @{username} 크롤링 완료")

    except Exception as e:
        print(f"❌ {user_id} 실패: {e}")

#driver.quit()

df = pd.DataFrame(results)
df.to_csv("x_user_data.csv", index=False, encoding='utf-8-sig')
print("📦 저장 완료: x_user_data.csv")

✅ 로그인 완료!
✅ @SNCmusic 크롤링 완료
✅ @mseckington 크롤링 완료
✅ @m_musterman12 크롤링 완료
✅ @Level39CW 크롤링 완료
✅ @RogueEconomist1 크롤링 완료
📦 저장 완료: x_user_data.csv


In [51]:
u2n("6014")

6014